In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import selenium
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd
import numpy as np
import os
import re
import lxml

In [2]:
driver = webdriver.Chrome("C:\\Users\\bobby\\Desktop\\Python_project\\chromedriver.exe")

In [3]:
driver.get("https://www.ufc.com/athletes/all")

In [4]:
ele=driver.find_element_by_id("onetrust-accept-btn-handler")

In [5]:
ele.click()

In [6]:
time.sleep(2)

In [7]:
total=BeautifulSoup(driver.page_source,"lxml")

In [8]:
totalAth=total.find_all("div", {"class": "althelete-total"})

In [9]:
stotal=re.search("\d+",str(totalAth)).group(0)
stotal=int(int(stotal)/11)


In [10]:
#for i in range(stotal):
#    ele=driver.find_element_by_class_name("pager__item")
#    ele.click()
#    time.sleep(10)

In [11]:
total=BeautifulSoup(driver.page_source,"lxml")

In [12]:
links=total.find_all("a", {"class": "e-button--black"})

In [13]:
ufc = "https://www.ufc.com"
links[0]["href"]
linksAH=pd.DataFrame(np.nan, index=range(len(links)), columns=["link"],dtype="string")

In [14]:
for i in range(len(links)):
        linksAH.at[i,'link']= ufc +str(links[i]["href"])
        

In [18]:
driver.get(linksAH.iloc[0,0])

In [19]:
time.sleep(2)

In [21]:
ele = driver.find_element_by_class_name("e-button--white")
ele.click()

while True:
    try:
        ele = driver.find_element_by_class_name("pager__item")
        ele.click()
    except:
        break

In [22]:
time.sleep(2)
stats = BeautifulSoup(driver.page_source,"lxml")

In [23]:
for a in stats.find_all(attrs={"class":"c-record__promoted"}):
    try:
        print(re.search("\d+",str(a)).group(0))
    except:
            print("0")                

9
4
7
0
0
0


In [24]:
for a in stats.find_all(attrs={"class":"e-chart-circle__percent"}):
    print(re.sub("<.*?>","",str(a)))

45%
24%


In [25]:
for a in stats.find_all(attrs ={"class":"c-overlap__stats-value"}):
    print(re.sub("<.*?>","",str(a)))

210
469
5
29


In [26]:
for a in stats.find_all(attrs={"class":"c-stat-compare__number"}):
    print(re.sub("<.*?>","",str(a)))

2.45
              
2.45
          
1.23
              
0.18
          
59
                  %
47
                      %
0.35
              
10:43
          


In [27]:
for a in stats.find_all(attrs={"class":"c-stat-3bar__value"}):
    print(re.sub("<.*?>","",str(a)))

129 (61%)
59 (28%) 
22 (10%)
9 (45%)
7 (35%) 
4 (20%)


In [28]:
print(stats.find(attrs={"id":"e-stat-body_x5F__x5F_head_value"}))
print(stats.find(attrs={"id":"e-stat-body_x5F__x5F_head_percent"}))
print(stats.find(attrs={"id":"e-stat-body_x5F__x5F_body_value"}))
print(stats.find(attrs={"id":"e-stat-body_x5F__x5F_body_percent"}))
print(stats.find(attrs={"id":"e-stat-body_x5F__x5F_leg_value"}))
print(stats.find(attrs={"id":"e-stat-body_x5F__x5F_leg_percent"}))


<text fill="#D20A0A" font-size="14px" id="e-stat-body_x5F__x5F_head_value" transform="matrix(1 0 0 1 197.8119 32)">131</text>
<text fill="#D20A0A" font-size="14px" id="e-stat-body_x5F__x5F_head_percent" transform="matrix(1 0 0 1 230.8119 32)">62%</text>
<text fill="#D20A0A" font-size="14px" id="e-stat-body_x5F__x5F_body_value" transform="matrix(1 0 0 1 197.8119 90)">37 </text>
<text fill="#D20A0A" font-size="14px" id="e-stat-body_x5F__x5F_body_percent" transform="matrix(1 0 0 1 226.8119 90)">18%</text>
<text fill="#D20A0A" font-size="14px" id="e-stat-body_x5F__x5F_leg_value" transform="matrix(1 0 0 1 197.8119 145)">42</text>
<text fill="#D20A0A" font-size="14px" id="e-stat-body_x5F__x5F_leg_percent" transform="matrix(1 0 0 1 226.8119 145)">20%</text>


In [29]:
for a in stats.find_all(attrs={"class":"c-bio__field"}):
    lab=a.find(attrs={"class":"c-bio__label"})
    lab = re.sub("<.*?>","",str(lab))
    c=re.sub("<.*?>","",str(a))
    c= re.search("\d+",str(c))
    if(lab=="Age"):
        print(c.group(0))
    if(lab=="Height"):
        print(c.group(0))
    if(lab=="Reach"):
        print(c.group(0))
    if(lab=="Leg reach"):
        print(c.group(0))

39
75
76
41


In [121]:
l= []

In [122]:
for a in stats.find_all(attrs={"class":"c-card-event--athlete-results__matchup"}):
    #if("Win"==re.sub("<.*?>","",str(a)).strip()):
    for j in a:
       l.append(j)
    

In [123]:
for i in range(len(l)):
    for a in range(len(l)):
        try:
            if str(l[a]) == "\n":
                l.pop(a)
        except:
            break


In [92]:
for a in stats.find_all(attrs={"class":"c-card-event--athlete-results__headline"}):
    print(re.sub("<.*?>","",str(a)))

Blaydes vs Abdurakhimov
Tybura vs Abdurakhimov
Arlovski vs Abdurakhimov
Sherman vs Abdurakhimov
Lewis vs Abdurakhimov
Abdurakhimov vs Harris
Hamilton vs Abdurakhimov
Abdurakhimov vs Johnson


[<div class="c-card-event--athlete-results__red-image">
 <div class="c-card-event--athlete-results__plaque win">
                                       Win
                                   </div>
 <article class="media media--type-image media--view-mode-event-results-athlete-headshot">
 <img class="image-style-event-results-athlete-headshot" height="160" src="https://dmxg5wxfqgb4u.cloudfront.net/styles/event_results_athlete_headshot/s3/2020-11/67683%252Fprofile-galery%252Fprofile-picture%252FBLAYDES_CURTIS_01-25.png?VersionId=null&amp;itok=qNRfs42h" width="256"/>
 </article>
 </div>,
 <div class="c-card-event--athlete-results__blue-image">
 <article class="media media--type-image media--view-mode-event-results-athlete-headshot">
 <img class="image-style-event-results-athlete-headshot" height="160" src="https://dmxg5wxfqgb4u.cloudfront.net/styles/event_results_athlete_headshot/s3/2018-10/ABDURAKHIMOV_SHAMIL.png?VersionId=JP8veq6flJTLFWMWCTYAkfez9OXYAe76&amp;itok=c9_ntfuv" width="256"/